In [1]:
import json
import matplotlib.pyplot as plt
import cv2
import os
import torch
import numpy as np
import fnmatch
import shutil
import glob
from scipy.io import loadmat


In [15]:
def xyxy2xywh(x):
    # Convert bounding box format from [x1, y1, x2, y2] to [x, y, w, h]
    # x, y are coordinates of center 
    # (x1, y1) and (x2, y2) are coordinates of bottom left and top right respectively. 
    y = torch.zeros_like(x) if x.dtype is torch.float32 else np.zeros_like(x)
    y[:, 0] = (x[:, 0] + x[:, 2]) / 2  # x center
    y[:, 1] = (x[:, 1] + x[:, 3]) / 2  # y center
    y[:, 2] = x[:, 2] - x[:, 0]  # width
    y[:, 3] = x[:, 3] - x[:, 1]  # height
    return y



In [4]:
## citypersons

path = './dataset/citypersons/labels/'
for i in os.listdir(path):
    for j in os.listdir(path+'/'+i):
        to_path = (path+'/'+i+'/'+j).replace('labels','labels_with_ids')
        if not os.path.exists(to_path):
            os.makedirs(to_path)

for i in os.listdir(path):
    for j in os.listdir(path+'/'+i):
        for n,k in enumerate(os.listdir(path+'/'+i+'/'+j)):
            json_path = (path+'/'+i+'/'+j+'/'+k)
            f = open(json_path, encoding='utf-8')
            data = json.load(f)
            txt_path = (path+'/'+i+'/'+j+'/'+k).replace('_gtBboxCityPersons.json','.txt').replace('labels','labels_with_ids')
            with open(txt_path, 'w+') as file:
                for obj in data['objects']:
                    if obj['instanceId']!=0:
                        bbox = obj['bbox'].copy()
                        bbox[0] = bbox[0] + bbox[2]/2
                        bbox[1] = bbox[1] + bbox[3]/2
                        file.write(str(n)+',-1,'+','.join(str(x) for x in bbox)+'\n')
                        # print(str(n)+',-1,'+','.join(str(x) for x in bbox))
            f.close()

In [45]:
## ETHZ

path = './dataset/ETH/images/'
for i in os.listdir(path):
    txt_path = path.replace('images', 'labels_with_ids')+i
    if not os.path.exists(txt_path):
        os.makedirs(txt_path)
    
path = './dataset/ETH/label/'
for i in os.listdir(path):
    txt_path = path+i
    folder_path = txt_path.replace('label', 'labels_with_ids').rstrip('.txt')
    with open(txt_path, 'r') as src:
        label_files = src.readlines()
        for n, label_file in enumerate(label_files):
            img_path = label_file[1:26].strip('left')
            cur_path = (folder_path + img_path).replace('.png', '.txt')
            if len(label_file)>29:
                bboxs = label_file[29:].replace(');\n', ')').split('), (')
                bboxs = [bbox.strip('(').strip(')') for bbox in bboxs]
                bboxs = [bbox.split(', ') for bbox in bboxs]
                bboxs = np.array(bboxs, dtype=np.int64)
                bboxs = xyxy2xywh(bboxs)
                label = np.zeros((bboxs.shape[0],2))
                label[:,0] = n
                label[:,1] = -1
                label = np.concatenate((label, bboxs), axis=1)
                np.savetxt(cur_path, label, fmt='%d', delimiter=',')
            else:
                dst = open(cur_path, 'w+')
                dst.close()

    



In [4]:
path = './dataset/ETH/labels_with_ids/Linthescher/'
for f in os.listdir(path):
    label = np.loadtxt(path+f, delimiter=',')
    if len(label) < 1:
        os.remove(path+f)

In [6]:
## CalTech images
rootdirs = os.listdir(".")
print(rootdirs)
# walk in the rootdir, take down the .seq filename and filepath
for rootdir in rootdirs:
    for parent, dirnames, filenames in os.walk(rootdir):
        for filename in filenames:
            # check .seq file with suffix
            if fnmatch.fnmatch(filename,'*.seq'):
                # take down the filename with path of .seq file
                thefilename = os.path.join(parent, filename)
                # create the image folder by combining .seq file path with .seq filename
                thesavepath = (parent+'/'+filename.split('.')[0]).replace('CT','CalTech')
                print("Filename=" + thefilename)
                print("Savepath=" + thesavepath)
                if not os.path.exists(thesavepath):
                    os.makedirs(thesavepath)
                vc = cv2.VideoCapture(thefilename)
                retval, frame = vc.read()
                i=0
                while (retval):
                    if (i%10==0):
                        cv2.imwrite(thesavepath+'/'+str(int(i/10)).rjust(6,'0')+'.jpg', frame)
                    retval, frame = vc.read()
                    i = i + 1


['__pycache__', 'data', 'dataset', 'dataset.ipynb', 'jde_rcnn.py', 'loaddataset.ipynb', 'model.ipynb', 'rename', 'test.py', 'utils']
Filename=dataset/CT/set00/V000.seq
Savepath=dataset/CalTech/set00/V000
Filename=dataset/CT/set00/V001.seq
Savepath=dataset/CalTech/set00/V001
Filename=dataset/CT/set00/V002.seq
Savepath=dataset/CalTech/set00/V002
Filename=dataset/CT/set00/V003.seq
Savepath=dataset/CalTech/set00/V003
Filename=dataset/CT/set00/V004.seq
Savepath=dataset/CalTech/set00/V004
Filename=dataset/CT/set00/V005.seq
Savepath=dataset/CalTech/set00/V005
Filename=dataset/CT/set00/V006.seq
Savepath=dataset/CalTech/set00/V006
Filename=dataset/CT/set00/V007.seq
Savepath=dataset/CalTech/set00/V007
Filename=dataset/CT/set00/V008.seq
Savepath=dataset/CalTech/set00/V008
Filename=dataset/CT/set00/V009.seq
Savepath=dataset/CalTech/set00/V009
Filename=dataset/CT/set00/V010.seq
Savepath=dataset/CalTech/set00/V010
Filename=dataset/CT/set00/V011.seq
Savepath=dataset/CalTech/set00/V011
Filename=datase

In [12]:
def renameFile(rootDir,filetype):
    for dir_or_file in os.listdir(rootDir):
        filePath = os.path.join(rootDir, dir_or_file)
        if os.path.isfile(filePath):
            if os.path.basename(filePath).endswith(filetype):
                # print('imgBox fileName is '+ filePath)
                basename = os.path.basename(filePath)
                tofilePath = filePath.rstrip(basename) + basename.rstrip(filetype).rjust(6,'0')+filetype
                os.rename(filePath, tofilePath)
            else:
                continue
        elif os.path.isdir(filePath):
            renameFile(filePath,filetype)
        else:print('not file and dir '+os.path.basename(filePath))
renameFile('./dataset/CUHK', '.jpg')

In [ ]:
## CalTech labels

all_obj = 0
label_path = './dataset/CalTech/labels_with_ids/'
for dname in sorted(glob.glob('./dataset/CT/annotations/set*')):
    set_name = os.path.basename(dname)
    for anno_fn in sorted(glob.glob('{}/*.vbb'.format(dname))):
        vbb = loadmat(anno_fn)
        objLists = vbb['A'][0][0][1][0]
        maxObj = int(vbb['A'][0][0][2][0][0])
        video_name = os.path.splitext(os.path.basename(anno_fn))[0]
        current_path = label_path+set_name+'/'+video_name
        n_obj = 0
        if not os.path.exists(current_path):
            os.makedirs(current_path)
        for frame_id, obj in enumerate(objLists):
            if frame_id%10==0:
                frame_id = int(frame_id / 10)
                with open(current_path+'/'+str(frame_id).rjust(6,'0')+'.txt', 'w+') as dst:
                    if len(obj) > 0:
                        for id, pos in zip(
                                obj['id'][0], obj['pos'][0]):
                            id = int(id[0][0]) - 1  # MATLAB is 1-origin
                            pos = pos[0].tolist()
                            pos[0] = pos[0] + pos[2]/2
                            pos[1] = pos[1] + pos[3]/2
                            dst.write(str(frame_id)+','+str(id)+','+','.join(str(x) for x in pos)+'\n')
                            # print(str(frame_id)+','+str(id)+','+','.join(str(x) for x in pos))
                            n_obj += 1

        print(dname, anno_fn, n_obj)
        all_obj += n_obj

print('Number of objects:', all_obj)

In [14]:
## MOT16

rootdir = './dataset/MOT16/train/'
for path in os.listdir(rootdir):
    path = rootdir+path
    gt = np.loadtxt(path+'/gt/gt.txt', dtype=int, delimiter=',', usecols=(0,1,2,3,4,5,6,7))
    gt = gt[np.argsort(gt[:,0])]
    gt = gt[gt[:,7]==1]
    # print(gt.shape)
    label = gt[:,1]
    for i,id in enumerate(np.unique(label)):
        label[label==id] = i
    gt[:,1] = label
    num_frames = np.unique(gt[:,0]).shape[0]
    current_path = path+'/labels_with_ids/'
    if not os.path.exists(current_path):
        os.makedirs(current_path)
    for i in range(num_frames):
        src = gt[gt[:,0]==i+1][:,0:6]
        src[:,2] = src[:,2] + src[:,4]/2
        src[:,3] = src[:,3] + src[:,5]/2
        np.savetxt(current_path+str(i+1).rjust(6,'0')+'.txt', src, fmt='%d', delimiter=',')


In [131]:
## CUHK images
rootdir = './dataset/CUHK/seq/'
filenames = os.listdir(rootdir)
for filename in filenames:
    thefilename = rootdir + filename
    # create the image folder by combining .seq file path with .seq filename
    thesavepath = thefilename.replace('/seq/', '/images/').rstrip('.seq')
    print("Filename=" + thefilename)
    print("Savepath=" + thesavepath)
    if not os.path.exists(thesavepath):
        os.makedirs(thesavepath)
    vc = cv2.VideoCapture(thefilename)
    retval, frame = vc.read()
    i=0
    while (retval):
        cv2.imwrite(thesavepath+'/'+str(int(i))+'.jpg', frame)
        retval, frame = vc.read()
        i = i + 1

Filename=./dataset/CUHK/seq/set00-occ.seq
Savepath=./dataset/CUHK/images/set00-occ
Filename=./dataset/CUHK/seq/set01-occ.seq
Savepath=./dataset/CUHK/images/set01-occ
Filename=./dataset/CUHK/seq/set02-occ.seq
Savepath=./dataset/CUHK/images/set02-occ
Filename=./dataset/CUHK/seq/set03-occ.seq
Savepath=./dataset/CUHK/images/set03-occ
Filename=./dataset/CUHK/seq/set04-occ.seq
Savepath=./dataset/CUHK/images/set04-occ
Filename=./dataset/CUHK/seq/set05-occ.seq
Savepath=./dataset/CUHK/images/set05-occ
Filename=./dataset/CUHK/seq/set06-occ.seq
Savepath=./dataset/CUHK/images/set06-occ
Filename=./dataset/CUHK/seq/set07-occ.seq
Savepath=./dataset/CUHK/images/set07-occ
Filename=./dataset/CUHK/seq/set08-occ.seq
Savepath=./dataset/CUHK/images/set08-occ


In [130]:
## CUHK labels

all_obj = 0
label_path = './dataset/CUHK/labels_with_ids/'
for anno_fn in os.listdir('./dataset/CUHK/gt/'):
    vbb = loadmat('./dataset/CUHK/gt/'+anno_fn)
    objLists = vbb['A'][0][0][1][0]
    maxObj = int(vbb['A'][0][0][2][0][0])
    current_path = label_path+anno_fn.rstrip('.vbb')
    n_obj = 0
    if not os.path.exists(current_path):
        os.makedirs(current_path)
    for frame_id, obj in enumerate(objLists):
        with open(current_path+'/'+str(frame_id)+'.txt', 'w+') as dst:
            if len(obj) > 0:
                for id, pos in zip(
                        obj['id'][0], obj['pos'][0]):
                    id = int(id[0][0]) - 1  # MATLAB is 1-origin
                    pos = pos[0].tolist()
                    pos[0] = pos[0] + pos[2]/2
                    pos[1] = pos[1] + pos[3]/2
                    dst.write(str(frame_id)+','+str(id)+','+','.join(str(x) for x in pos)+'\n')
                    # print(str(frame_id)+','+str(id)+','+','.join(str(x) for x in pos))
                    n_obj += 1

    all_obj += n_obj

print('Number of objects:', all_obj)

Number of objects: 10191


In [25]:
# PRW

src = '/mnt/d/Study/graduation_project/dataset/PRW/annotations/'
dst = './dataset/PRW/labels_with_ids/'
if not os.path.exists(dst):
    os.makedirs(dst)
for i,basename in enumerate(os.listdir(src)):
    mat = loadmat(src+basename)
    try:
        label = mat['box_new']
    except:
        try:
            label = mat['anno_file']
        except:
            label = mat['anno_previous']
    frame_id = np.ones((label.shape[0],1))*(i+1)
    label = np.concatenate((frame_id, label), axis=1)
    label[:,1] = label[:,1] - 1
    label[:,2] = label[:,2] + label[:,4]/2
    label[:,3] = label[:,3] + label[:,5]/2
    label[label==-3] = -1
    save_path = dst + basename.replace('.jpg.mat', '.txt')
    np.savetxt(save_path, label, fmt='%f', delimiter=',')

In [26]:
## check id
path = './data/PRW.txt'
with open(path, 'r') as file:
    img_files = file.readlines()
    img_files = [x.replace('\n', '') for x in img_files]
    img_files = list(filter(lambda x: len(x) > 0, img_files))

label_files = [x.replace('images', 'labels_with_ids').replace('.png', '.txt').replace('.jpg', '.txt')
                    for x in img_files]

ids = []
for lp in label_files:
    lb = np.loadtxt(lp, delimiter=',')
    if len(lb) < 1:
        continue
    if len(lb.shape) < 2:
        ids.append(lb[1])
    else:
        ids = ids + list(lb[:,1])

set(ids)



{-1.0,
 0.0,
 1.0,
 2.0,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.0,
 9.0,
 10.0,
 11.0,
 12.0,
 13.0,
 14.0,
 15.0,
 16.0,
 17.0,
 18.0,
 19.0,
 20.0,
 21.0,
 22.0,
 23.0,
 24.0,
 25.0,
 26.0,
 27.0,
 28.0,
 29.0,
 30.0,
 31.0,
 32.0,
 33.0,
 34.0,
 35.0,
 36.0,
 37.0,
 38.0,
 39.0,
 40.0,
 41.0,
 42.0,
 43.0,
 44.0,
 45.0,
 46.0,
 47.0,
 48.0,
 49.0,
 50.0,
 51.0,
 52.0,
 53.0,
 54.0,
 55.0,
 56.0,
 57.0,
 58.0,
 59.0,
 60.0,
 61.0,
 62.0,
 63.0,
 64.0,
 65.0,
 66.0,
 67.0,
 68.0,
 69.0,
 70.0,
 71.0,
 72.0,
 73.0,
 74.0,
 75.0,
 76.0,
 77.0,
 78.0,
 79.0,
 80.0,
 81.0,
 82.0,
 83.0,
 84.0,
 85.0,
 86.0,
 87.0,
 88.0,
 89.0,
 90.0,
 91.0,
 92.0,
 93.0,
 94.0,
 95.0,
 96.0,
 97.0,
 98.0,
 99.0,
 100.0,
 101.0,
 102.0,
 103.0,
 104.0,
 105.0,
 106.0,
 107.0,
 108.0,
 109.0,
 110.0,
 111.0,
 112.0,
 113.0,
 114.0,
 115.0,
 116.0,
 117.0,
 118.0,
 119.0,
 120.0,
 121.0,
 122.0,
 123.0,
 124.0,
 125.0,
 126.0,
 127.0,
 128.0,
 129.0,
 130.0,
 131.0,
 132.0,
 133.0,
 134.0,
 135.0,
 136.0,
 137.0,

In [10]:
len(set(ids))

5071

In [17]:
a = np.ones((4,3))
a[:,1] = -2
a

array([[ 1., -2.,  1.],
       [ 1., -2.,  1.],
       [ 1., -2.,  1.],
       [ 1., -2.,  1.]])

In [22]:
label

array([[11816.        ,   886.        ,    62.17088608,   393.61392405,
          117.87341772,   364.55696203]])